## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification French

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `fr.deid_obfuscated`
- **Model Description**: This pipeline can be used to deidentify PHI information from medical texts in French. The pipeline can mask and obfuscate the following entities: DATE, AGE, SEX, PROFESSION, ORGANIZATION, PHONE, E-MAIL, ZIP, STREET, CITY, COUNTRY, PATIENT, DOCTOR, HOSPITAL, MEDICALRECORD, SSN, IDNUM, ACCOUNT, PLATE, USERNAME, URL, and IPADDR.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "fr-deid-obfuscated"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w') as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data)
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


### Initial Setup

In [7]:
docs = [
'''PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058, ADRESSE : 18 Avenue Matabiau, VILLE : Grenoble
CODE POSTAL : 38000 DATE DE NAISSANCE : 03/03/1946, Âge : 70 ans , Sexe : H, NUMÉRO DE TÉLÉPHONE : 06-777413307, COURRIEL : jdubois@hotmail.fr, DATE D'ADMISSION : 12/12/2016
MÉDÉCIN : Dr Michel Renaud, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
Jean Dubois nous a été adressé car il présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV.
L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.
Hémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml.
ADDRESSÉ À : Dre Marie Breton - Centre Hospitalier de Bellevue Service D'Endocrinologie et de Nutrition - Rue Paulin Bussières, 38000Grenoble
COURRIEL : mariebreton@chb.fr
NUMÉRO DE TÉLÉPHONE : 06-34567890''',

'''PRENOM : Éric, NOM : Lejeune, NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020, ADRESSE : 74 Boulevard Riou, VILLE : Sainte Annenec
CODE POSTAL : 49722 DATE DE NAISSANCE : 10/06/1980, Âge : 38 ans , Sexe : H, NUMÉRO DE TÉLÉPHONE : 06-90123456. COURRIEL : elej@free.fr, DATE D'ADMISSION : 08/11/2018
MÉDÉCIN : Dre Martine Da Costa, RAPPORT CLINIQUE : 
Antécédents : Éric Lejeune de 38 ans, évalué en consultation pour une coxalgie droite d'un an et demi d'évolution, de caractéristiques mécaniques. Éric Lejeune a signalé une douleur à l'exercice et une légère limitation de la mobilité dans certaines postures. Mécanicien de profession, pendant son temps libre, Éric Lejeune faisait du vélo (40 km par jour le week-end), courait 3 fois par semaine pendant 50 minutes et marchait pendant une heure les jours où il ne courait pas. La douleur a progressivement augmenté et elle ne fait actuellement que marcher.
EXAMEN PHYSIQUE : L'examen physique a révélé une douleur à la palpation dans le tiers moyen de la région inguinale droite avec une douleur à la flexion, à l'adduction et à la rotation, surtout à la rotation interne (manœuvre de choc positif). L'équilibre de l'articulation de la hanche est de 110o de flexion bilatérale, 10o de rotation interne droite, 40o de rotation interne gauche et 30o de rotation externe droite, 40o de rotation externe gauche. L'équilibre musculaire sur l'échelle de Daniels est 5/5 global bilatéral. Pas de douleur à la palpation des points sciatiques et les manœuvres de Valleix, Fabere, Lasegue et Bragard sont négatives. Pas non plus de douleur à la palpation de la bandelette iliotibiale et le test d'Ober était négatif. A l'examen de la marche, nous avons observé une absence de claudication et la posture monopodale alternée droite-gauche était stable, avec un Trendelenburg négatif. Le patient avait des douleurs lors de la marche avec une flexion forcée de la hanche et du genou.
Examens complémentaires : Une radiologie ordinaire antéropostérieure (AP) a montré une déformation en forme de bosse dans la région fémorale cervicale droite. L'hémogramme, la biochimie, les hormones thyroïdiennes, les anticorps et les réactifs de phase aiguë requis étaient normaux.
Au vu de ces résultats, une IRM de la hanche a été demandée, qui a montré de légères modifications sous-chondrales, avec un œdème osseux dans la hanche droite, avec une irrégularité corticale et une légère synovite dans l'interligne articulaire, avec une distension de la bourse ilio-psoas. La tête fémorale présente un petit îlot osseux et l'angle alpha est supérieur à 50o. L'arthrographie montre un pincement de l'interligne articulaire antéro-supérieur et postéro-inférieur, un col fémoral ostéophytique naissant avec de petites lésions sous-chondrales antéro-supérieures de la tête fémorale et de petits foyers d'œdème dans l'acétabulum antéro-supérieur. Pas de nécrose avasculaire ni de lésions du labrum. Une lésion kystique juxta-articulaire est également observée en regard du grand trochanter.
Evolution : Avec le diagnostic de syndrome de conflit fémoroacétabulaire de type LEVA (également connu sous le nom de CAM), le patient s'est vu prescrire un traitement par magnétothérapie (15 séances) pour améliorer les symptômes de la douleur et le trophisme osseux.
Éric Lejeune a été adressé à l'unité d'arthroscopie du service de traumatologie pour une évaluation de la chirurgie arthroscopique.
Diagnostic principal : syndrome du conflit fémoroacétabulaire de type LEVA.
TRAITEMENT : Un traitement pharmacologique est prescrit avec une combinaison de glucosamine-chondroïtine sulfate par voie orale 400 mg toutes les 12 heures pendant 3 mois et des anti-inflammatoires non stéroïdiens si nécessaire en cas de douleur. Elle a pour consigne de se reposer dans les sports pour éviter tout choc à l'articulation, avec une activité active contrôlée, en recommandant de ne pas dépasser 90 degrés de flexion de la hanche et d'éviter la rotation. Prenez rendez-vous pour un suivi une fois les sulfates de glucosamine-chondroïtine terminés afin de constater une amélioration clinique. Appelez le numéro des soins et des rendez-vous précédents de l'hôpital : 0470892301.
ADDRESSÉ À : Pr Diane Martinez - Centre Hospitalier Saint Louis - 30 Rue Garcia, 49000Angers
COURRIEL : dmartinez@chsl.fr
NUMÉRO DE TÉLÉPHONE : 06-44006512'''

]


sample_text = """NOM : PerAurélieMÉRO DEPerrierTÉ SOCIALE : 2190150502008
ADR2190150502008Faivre
VIL.8 Rue Faivre Aurélie
Bousquetrrier
NUMÉRO DE11631ITÉ SOCIALE : 21901522/10/1960RESSE :44 ans Faivre
VFemme: Bousquet
Caurelieprr@orange.frATE DE NAISSANCE : 231/12/2004Âge : 44 anDr Alain Bouvet
COURRIEL : aurelieprr@orange.fr
DATE D'ADMIAurélie Perrier004
MÉDÉC44: Dr Alain Bouvet
RAPPORT CLINIQUE : 
Motif de l'admission : Aurélie Perrier âgée de 44 ans, adressée à notre unité en raison d'une augmentation de volume dans la région Lesomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Les antécédents médicaux n'étaient pas pertinents. 
Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde.
Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures.
L'étude histopathologique Lesignalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. Les contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée Dr Joseph Pasquier reCentre Hospitalier de Carcassonne - 1060 Chemin de la MadeleineÀ 11000Carcassonneier - Centre Hospitalie06-71105388sonne - 1060pasquier@orange.frleine, 11000 Carcassonne
NUMÉRO DE TÉLÉPHONE : 06-71105388
COURRIEL : pasquier@orange.fr"""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**PRENOM : Éric, NOM : Lejeune, NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020, ADRESSE : 74 Boulevard Riou, VILLE : Sainte Annenec**"

    - **masked**: Default policy that masks entities with their type.

      -> 'PRENOM : `<PATIENT>`, NOM : `<PATIENT>`, NUMÉRO DE SÉCURITÉ SOCIALE : `<SSN>`, ADRESSE : `<STREET>`, VILLE : `<CITY>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'PRENOM : `Mlle Goncalves`, NOM : `Mme Garnier-Roussel`, NUMÉRO DE SÉCURITÉ SOCIALE : `129022554007724`, ADRESSE : `avenue de Mahe`, VILLE : `Meyer-sur-Lemoine`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'PRENOM : `****`, NOM : `****`, NUMÉRO DE SÉCURITÉ SOCIALE : `****`, ADRESSE : `****`, VILLE : `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'PRENOM : [`**`], NOM : [`*****`], NUMÉRO DE SÉCURITÉ SOCIALE : [`***********`], ADRESSE : [`***************`], VILLE : [`************`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}


### **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
# masked (default-policy)
data_dicts = [
    {
        "text": sample_text
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"NOM : PerAurélieMÉRO DEPerrierTÉ SOCIALE : <SSN> <MEDICALRECORD>\nVIL. <STREET> <CITY> NUMÉRO DE11631ITÉ SOCIALE : <MEDICALRECORD> :<AGE> <COUNTRY> V<SEX>: <PATIENT> <E-MAIL>. frATE DE NAISSANCE : 2<DATE>/2004Âge : <AGE> COURRIEL : <E-MAIL>\nDATE <USERNAME> <PHONE> MÉDÉC44: <DOCTOR> RAPPORT CLINIQUE : \nMotif de l'admission : <PATIENT> âgée de <AGE>ans, adressée à notre unité en raison d'une augmentation de volume dans la région Lesomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. <CITY> antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde. Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures. L'étude histopathologique Lesignalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. <CITY> contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée <DOCTOR> reCentre <HOSPITAL> - 1060 Chemin de la MadeleineÀ 11000Carcassonneier - Centre Hospitalie<PHONE>sonne - <E-MAIL>. frleine, <ZIP><CITY> NUMÉRO DE TÉLÉPHONE : <PHONE> COURRIEL : <E-MAIL>. fr"


In [9]:
# obfuscated
data_dicts = [
    {
        "text": sample_text,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"NOM : PerAurélieMÉRO DEPerrierTÉ SOCIALE : 1610960454098 JXB1478295621308MVHQIO\nVIL. 91, rue de De Sousa Sainte Sabine-les-Bains NUMÉRO DE11631ITÉ SOCIALE : 96295284/13/2440NUUVO :46 Guyane VHomme: M. Zacharie Ollivier eugenedavid@club-internet.fr. frATE DE NAISSANCE : 231/12/2004Âge : 36 COURRIEL : margotpichon@ifrance.com\nDATE didiergilles ZDGUYQI347 MÉDÉC44: Dr. Nicolas RAPPORT CLINIQUE : \nMotif de l'admission : Mme Thibaut du Hebert âgée de 56ans, adressée à notre unité en raison d'une augmentation de volume dans la région Lesomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Roger antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde. Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures. L'étude histopathologique Lesignalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. Roger contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée Dre. Claude-Margaux Charrier reCentre Centre Hospitalier De Bar Sur Aube - 1060 Chemin de la MadeleineÀ 11000Carcassonneier - Centre Hospitalie42-59563875sonne - denis50@noos.fr. frleine, 64332Lacombe NUMÉRO DE TÉLÉPHONE : 95-18841660 COURRIEL : weissolivier@noos.fr. fr"


### **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [10]:
# masked (default-policy)
data_dicts = [
    {
        "text": docs
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         PRENOM : Jean, NOM : <PATIENT>, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : <STREET>, VILLE : <CITY> CODE POSTAL : <ZIP>DATE DE NAISSANCE : <DATE>, Âge : <AGE> , Sexe : <SEX>, NUMÉRO DE TÉLÉPHONE : <PHONE>, COURRIEL : <E-MAIL>, DATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR>, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. <PATIENT> nous a été adressé car <SEX> présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.\nHémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; le

In [11]:
# obfuscated
data_dicts = [
    {
        "text": docs,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            PRENOM : Jean, NOM : Mme Marie Lamy, NUMÉRO DE SÉCURITÉ SOCIALE : 1610960454098, ADRESSE : boulevard de Vallet, VILLE : Sainte Antoine CODE POSTAL : 11914DATE DE NAISSANCE : 03/03/1946, Âge : 46 , Sexe : Femme, NUMÉRO DE TÉLÉPHONE : 78-295621308, COURRIEL : georgeslemonnier@live.com, DATE D'ADMISSION : 12/12/2016 MÉDÉCIN : Dr. Sauvage, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. Mme Alexandrie Brun nous a été adressé car Homme présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 glo

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import os

validation_file_name_1 = "input_1.json"
validation_file_name_2 = "input_2.json"
validation_file_name_3 = "input_3.json"
validation_file_name_4 = "input_4.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [14]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data)

    with open(file_name, "w") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )

In [15]:
# Define input JSON data for each validation file
input_json_data = {
    validation_file_name_1: {"text": docs},
    validation_file_name_2: {"text": docs, "masking_policy": "obfuscated"},
    validation_file_name_3: {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    validation_file_name_4: {"text": docs, "masking_policy": "masked_with_chars"},
}

# Write and upload each input JSON data to S3
for file_name, json_data in input_json_data.items():
    write_and_upload_to_s3(json_data, f"{input_dir}/{file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type= batch_transform_inference_instance_type,
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [17]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

#### masked (default-policy)

In [18]:
process_s3_output_and_save(validation_file_name_1, "out_1.out")

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         PRENOM : Jean, NOM : <PATIENT>, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : <STREET>, VILLE : <CITY> CODE POSTAL : <ZIP>DATE DE NAISSANCE : <DATE>, Âge : <AGE> , Sexe : <SEX>, NUMÉRO DE TÉLÉPHONE : <PHONE>, COURRIEL : <E-MAIL>, DATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR>, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. <PATIENT> nous a été adressé car <SEX> présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.\nHémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; le

#### obfuscated

In [19]:
process_s3_output_and_save(validation_file_name_2, "out_2.out")

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            PRENOM : Jean, NOM : Mme Marie Lamy, NUMÉRO DE SÉCURITÉ SOCIALE : 1610960454098, ADRESSE : boulevard de Vallet, VILLE : Sainte Antoine CODE POSTAL : 11914DATE DE NAISSANCE : 03/03/1946, Âge : 46 , Sexe : Femme, NUMÉRO DE TÉLÉPHONE : 78-295621308, COURRIEL : georgeslemonnier@live.com, DATE D'ADMISSION : 12/12/2016 MÉDÉCIN : Dr. Sauvage, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. Mme Alexandrie Brun nous a été adressé car Homme présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 glo

#### masked_fixed_length_chars

In [20]:
process_s3_output_and_save(validation_file_name_3, "out_3.out")

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               PRENOM : Jean, NOM : ****, NUMÉRO DE SÉCURITÉ SOCIALE : ****, ADRESSE : ****, VILLE : **** CODE POSTAL : ****DATE DE NAISSANCE : ****, Âge : **** , Sexe : ****, NUMÉRO DE TÉLÉPHONE : ****, COURRIEL : ****, DATE D'ADMISSION : **** MÉDÉCIN : ****, RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. **** nous a été adressé car **** présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.\nHémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml. ADDRESS

#### masked_with_chars

In [21]:
process_s3_output_and_save(validation_file_name_4, "out_4.out")

predictions
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              PRENOM : Jean, NOM : [****], NUMÉRO DE SÉCURITÉ SOCIALE : [***********], ADRESSE : [****************], VILLE : [******] CODE POSTAL : [***]DATE DE NAISSANCE : [********], Âge : [****] , Sexe : *, NUMÉRO DE TÉLÉPHONE : [**********], COURRIEL : [****************], DATE D'ADMISSION : [********] MÉDÉCIN : [**************], RAPPORT CLINIQUE : 70ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. [*********] nous a été adressé car ** présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV. L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.\nHémogramme normal ; la biochimie a montré

In [22]:
model.delete_model()

INFO:sagemaker:Deleting model with name: fr-deid-obfuscated-2024-03-22-07-51-44-822


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

